In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Overview

In [ ]:
df = pd.read_csv("transport_shipment_data.csv")
df = df.drop('Product Id', axis=1)
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.info()

## Handling Missing Values

In [ ]:
missing_data = df.isnull().sum()
missing_data = missing_data[missing_data > 0]
missing_data

In [ ]:
missing_data_proportion = df.isnull().mean() * 100
missing_data_proportion = missing_data_proportion[missing_data_proportion > 0]
missing_data_proportion

Expiry Period (5.00%): Since the percentage of missing data is low, imputation would also be a simple solution, using the mean or median.

Perishable Index (2.35%): With such a low proportion of missing data, imputation would also be a simple solution, using the mean or median.

F992 (37.30%):With a high proportion of missing values, you have two main options:
	•	Imputation: You could use more advanced imputation techniques such as KNN imputation or predictive imputation, though basic methods (like the mean) could also work if it’s important to keep this feature.
	•	Dropping the Column: If this feature isn’t essential or strongly correlated with your target variable, dropping it might be a viable option to avoid introducing noise.

In [ ]:
# Calculate the correlation between F992 and the transport mode variables (Air, Road, Rail, Sea)
correlation_with_target = df[['F992', 'Air', 'Road', 'Rail', 'Sea']].corr()
# Display the correlation values between F992 and each of the transport mode columns
correlation_with_target

F992 vs. Air: -0.037 (very weak negative correlation)
F992 vs. Road: -0.293 (moderate negative correlation)
F992 vs. Rail: 0.359 (moderate positive correlation)
F992 vs. Sea: 0.124 (weak positive correlation)

F992 has a moderate positive correlation with Rail (0.359) and a moderate negative correlation with Road (-0.293). These suggest that F992 has some influence when predicting Rail and Road transport modes.
	•	The correlations with Air and Sea are quite weak, indicating F992 may not be as important for predicting those modes.

In [ ]:
df['F992'].fillna(df['F992'].mean(), inplace=True)
df['Expiry Period'].fillna(df['Expiry Period'].mean(), inplace=True)
df['Perishable Index'].fillna(df['Perishable Index'].mean(), inplace=True)
missing_data = df.isnull().sum()
missing_data = missing_data[missing_data > 0]
missing_data

## Target Variable Analysis

In [ ]:
# Checking the distribution of the target variables (Air, Road, Rail, Sea) using .value_counts()
target_distribution = df[['Air', 'Road', 'Rail', 'Sea']].sum()

plt.figure(figsize=(18, 6))
target_distribution.plot(kind='bar', color='blue', alpha=0.5)
plt.title('Distribution of Transport Mode Classes')
plt.ylabel('Count')
plt.xlabel('Transport Mode')
plt.xticks(rotation=0)
plt.show()
target_distribution

There is still some imbalance (especially with Road having significantly more instances than the other classes), it is not as extreme. Some models (like RandomForest or XGBoost) are more robust to class imbalances and may not require resampling.

Support Vector Machines (SVM) with One-vs-Rest can work well with high-dimensional data and perform well in binary classification tasks but computationally expensive for large datasets.
Logistic Regression with One-vs-Rest (OvR) can be a strong baseline model, especially when using One-vs-Rest for multiclass classification but may underperform if the relationships between features are non-linear or complex. SMOTE or oversampling techniques may be needed if using models that do not naturally handle imbalance well (Logistic Regression, SVM)

LGBMClassifier and XGBoostClassifier are being chosen as the models to be trained as they handle class imbalance through their objective functions, perform well on structured data and are efficient for large datasets.


## Univariate Analysis

In [ ]:
categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_features = df.select_dtypes(include=['number']).columns.tolist()

print("Categorical Features:")
print(categorical_features)

print("\nDistinct values for each categorical feature:")
for feature in categorical_features:
    distinct_values = df[feature].nunique()
    unique_values = df[feature].unique()
    print(f"{feature}: {distinct_values} unique values")
    print(f"Values: {unique_values}")

print("\nNumerical Features:")
print(numerical_features)

In [ ]:
# Univariate Analysis - Numerical Features
numerical_columns = ['Net Weight', 'Value', 'Packaging Cost', 'Expiry Period', 'Length', 
                     'Height', 'Width', 'Volume', 'Perishable Index', 'Flammability Index', 'F145', 'F7987', 'F992']

# Plotting histograms and box plots for numerical features
plt.figure(figsize=(20, 15))

for i, col in enumerate(numerical_columns):
    plt.subplot(5, 4, i+1)
    sns.histplot(df[col], kde=True, color='blue')
    plt.title(f'Histogram and KDE for {col}')
    
plt.tight_layout()
plt.show()

In [ ]:
# Categorical Features (Size)
categorical_columns = ['Size']
plt.figure(figsize=(6, 4))
sns.countplot(x='Size', data=df, color='blue', alpha=0.5)
plt.title('Bar Plot for Size')
plt.show()

### Outlier Analysis

In [ ]:
plt.figure(figsize=(20, 15))

for i, col in enumerate(numerical_columns):
    plt.subplot(5, 4, i+1)
    sns.boxplot(x=df[col], color='skyblue')
    plt.title(f'Box Plot for {col}')
    
plt.tight_layout()
plt.show()


In [ ]:
def detect_outliers_iqr(df, columns):
    outliers = {}
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    return outliers

outliers = detect_outliers_iqr(df, numerical_columns)
outliers

No outliers detected for most of the numerical columns like Net Weight, Value, Packaging Cost, Expiry Period, etc.
Column F7987: 207 outliers detected.
Column F992: 361 outliers detected.

In [ ]:
# Calculate skewness for numerical columns
skewness = df[numerical_columns].skew()
print(skewness)

Symmetrical: Features with skewness close to 0 (between -0.5 and 0.5) are considered roughly symmetric, meaning they are not significantly skewed.
	•	Net Weight: -0.014 (very symmetric)
	•	Value: 0.283 (slightly positive skew)
	•	Packaging Cost: 0.179 (slightly positive skew)
	•	Expiry Period: -0.279 (slightly negative skew)
	•	Length: 0.036 (very symmetric)
	•	Height: 0.263 (slightly positive skew)
	•	Width: -0.439 (slightly negative skew)
	•	Volume: -0.163 (slightly negative skew)
	•	Perishable Index: -0.0008 (very symmetric)
	•	Flammability Index: 0.021 (very symmetric)
	•	F145: 0.0069 (very symmetric)
	•	F992: -0.211 (slightly negative skew)

Moderate Skew: A few features exhibit moderate skewness (between 0.5 and 1 or -1 and -0.5):
	•	F7987: 0.305 (slightly positive skew)

## Multivariate Analysis

### Multicollinearity Check

In [ ]:
numerical_df = df[numerical_columns]
pearson_corr_matrix = numerical_df.corr(method='pearson')
spearman_corr_matrix = numerical_df.corr(method='spearman')

print("Pearson Correlation Matrix:")
print(pearson_corr_matrix)

print("\nSpearman Correlation Matrix:")
print(spearman_corr_matrix)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.heatmap(pearson_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', ax=axes[0])
axes[0].set_title('Pearson Correlation Matrix')
sns.heatmap(spearman_corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', ax=axes[1])
axes[1].set_title('Spearman Correlation Matrix')
plt.tight_layout()
plt.show()

### Correlation analysis with the target variable
Checking the correlation between each feature and the individual target classes (Air, Road, Rail, Sea) using Pearson or Spearman correlations for numerical targets, and techniques like ANOVA or Chi-square for categorical targets.

In [ ]:
# Correlation of numerical features with all 4 target variables (Air, Road, Rail, Sea)
correlation_with_targets = {}

for target in ['Air', 'Road', 'Rail', 'Sea']:
    correlation_with_targets[target] = df[numerical_columns].corrwith(df[target])

correlation_with_targets_df = pd.DataFrame(correlation_with_targets)
correlation_with_targets_df

Insights:
	•	Net Weight shows a moderate positive correlation with Rail (0.374) and a negative correlation with Road (-0.355).
	•	Value is positively correlated with Rail (0.475), but negatively correlated with Air (-0.343).
	•	Height has a strong positive correlation with Air (0.418) and a negative correlation with Rail (-0.220).
	•	Width is positively correlated with Sea (0.276), while negatively correlated with Road (-0.306).
	•	Packaging Cost shows a positive correlation with Road (0.281) but is negatively correlated with Rail and Sea.

Conclusion:
	•	Features like Net Weight, Value, Height, and Width appear to have the most influence on the different transport modes.
	•	Height and Net Weight are especially important for Air and Rail.

In [ ]:
categorical_columns = ['Size']
chi_square_results = {}

for target in ['Air', 'Road', 'Rail', 'Sea']:
    for col in categorical_columns:
        contingency_table = pd.crosstab(df[col], df[target])
        chi2, p, dof, ex = chi2_contingency(contingency_table)
        chi_square_results[f'{col} vs {target}'] = p

chi_square_df = pd.DataFrame(chi_square_results.items(), columns=['Feature vs Target', 'p-value'])
chi_square_df

p-value > 0.05: For all target variables, the p-values are greater than 0.05, meaning that the feature Size does not show a statistically significant association with any of the target variables. This suggests that Size may not be a strong predictor of the transport mode (Air, Road, Rail, Sea)

## Feature Encoding
Categorical Features are encoded.Target Variables (Air, Road, Rail, Sea) are already represented as binary (0/1), so no encoding is required for the target.

In [ ]:
label_encoder = LabelEncoder()
df['Size'] = label_encoder.fit_transform(df['Size'])
df.head()

## Model Development

In [ ]:
# Train models using StratifiedKFold
def train_models(models_dict, X, y_labels, skf, evaluation_reports, cv_f1_scores):
    for model_name, model in models_dict.items():
        for label in y_labels:

            fold_f1_scores = []  # Store F1 scores for each fold
            fold_reports = []  # Store classification reports for each fold

            # Stratified K-Fold cross-validation
            for train_index, test_index in skf.split(X, df[label]):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = df[label].iloc[train_index], df[label].iloc[test_index]

                model.fit(X_train, y_train)  # Train the model

                # Predict the target on the test set
                y_pred = model.predict(X_test)

                # Calculate weighted F1 score and store for each fold
                fold_f1 = f1_score(y_test, y_pred, average='weighted')
                fold_f1_scores.append(fold_f1)

                # Store the classification report
                try:
                    report = classification_report(y_test, y_pred, output_dict=True)
                    fold_reports.append(report)
                except ValueError:
                    fold_reports.append({})  # Add an empty dict if the report fails

            # Store F1 scores and reports for each label
            cv_f1_scores[model_name][label] = fold_f1_scores
            evaluation_reports[model_name][label] = fold_reports

# Calculate the mean F1 scores across folds
def calculate_mean_f1_scores(cv_f1_scores, y_labels):
    mean_f1_scores_df = pd.DataFrame({
        model_name: {label: np.mean(cv_f1_scores[model_name][label]) for label in y_labels}
        for model_name in models_dict.keys()
    })
    return mean_f1_scores_df

def average_metrics_across_folds(reports):
    """Compute the average metrics across folds."""
    avg_report = {
        '0': {'precision': [], 'recall': [], 'f1-score': [], 'support': []},
        '1': {'precision': [], 'recall': [], 'f1-score': [], 'support': []},
        'accuracy': []
    }

    # Accumulate metrics for each fold
    for report in reports:
        for cls in ['0', '1']:
            avg_report[cls]['precision'].append(report[cls]['precision'])
            avg_report[cls]['recall'].append(report[cls]['recall'])
            avg_report[cls]['f1-score'].append(report[cls]['f1-score'])
            avg_report[cls]['support'].append(report[cls]['support'])
        avg_report['accuracy'].append(report['accuracy'])
    
    # Compute average values, using np.nanmean to handle potential NaN values
    for cls in ['0', '1']:
        avg_report[cls]['precision'] = np.nanmean(avg_report[cls]['precision'])
        avg_report[cls]['recall'] = np.nanmean(avg_report[cls]['recall'])
        avg_report[cls]['f1-score'] = np.nanmean(avg_report[cls]['f1-score'])
        avg_report[cls]['support'] = np.nanmean(avg_report[cls]['support'])
    avg_report['accuracy'] = np.nanmean(avg_report['accuracy'])
    
    return avg_report

def generate_classification_report_tables(evaluation_reports):
    """Generate and display tables with averaged metrics across folds."""
    for model_name, labels in evaluation_reports.items():
        for label_name, reports in labels.items():
            print(f"\n=== {model_name} - {label_name} Label ===")
            
            if reports:
                # Average out the metrics across the folds
                avg_report = average_metrics_across_folds(reports)
                
                # Create the classification report table using the averaged report
                table = create_classification_report_table(avg_report, model_name, label_name)
                
                if not table.empty:
                    print(table.to_string(index=False))
                else:
                    print(f"Classification report for {model_name} - {label_name} could not be displayed.")
            else:
                print(f"No reports found for {model_name} - {label_name}")

def create_classification_report_table(report_dict, model_name, label_name):
    """Create a DataFrame from the averaged classification report."""
    # Extract class 0 and class 1 metrics
    class_0 = report_dict['0']
    class_1 = report_dict['1']
    
    # Create DataFrame
    df = pd.DataFrame({
        'Metric': ['Precision', 'Recall', 'F1-score', 'Support'],
        f'{model_name} (Class 0)': [class_0['precision'], class_0['recall'], class_0['f1-score'], class_0['support']],
        f'{model_name} (Class 1)': [class_1['precision'], class_1['recall'], class_1['f1-score'], class_1['support']],
    })
    
    # Adding accuracy as a separate row
    df = pd.concat([df, pd.DataFrame({
        'Metric': ['Accuracy'],
        f'{model_name} (Class 0)': [report_dict['accuracy']],
        f'{model_name} (Class 1)': [np.nan]  # Accuracy is global, not per class
    })], ignore_index=True)
    
    return df

def plot_comparison(metric, evaluation_dfs):
    """
    Plots the comparison of the specified metric across models for each label.
    The metric can be 'accuracy', 'precision', 'recall', or 'f1-score'.
    """
    plt.figure(figsize=(10, 6))
    
    for model_name, labels in evaluation_dfs.items():
        metric_values = []  # To store the metric values for each label
        
        for label in y_labels:
            # Get the first report (as the example assumes we are working with the first fold's report)
            reports_for_label = labels[label]
            
            if reports_for_label:
                report = reports_for_label[0]  # Use the first fold's report
                
                if metric == 'accuracy':
                    # Directly get the accuracy metric from the report
                    metric_value = report['accuracy']
                else:
                    # For other metrics like precision, recall, f1-score, get the class 0 average
                    metric_value = report['0'][metric]
                
                metric_values.append(metric_value)
            else:
                metric_values.append(None)  # If there's no report for this label
        
        plt.plot(y_labels, metric_values, marker='o', label=model_name)
    
    plt.title(f'Comparison of {metric} across models')
    plt.xlabel('Labels')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.show()

In [ ]:
# Define the models to be trained
models_dict = {
    'LightGBM': lgb.LGBMClassifier(class_weight='balanced', verbose=-1),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', verbosity=0),
    'CatBoost': CatBoostClassifier(verbose=0)
}

# Initialize evaluation dictionaries to store cross-validation reports and F1 scores
evaluation_reports = {model_name: {label: [] for label in ['Air', 'Road', 'Rail', 'Sea']} for model_name in models_dict.keys()}
cv_f1_scores = {model_name: {label: [] for label in ['Air', 'Road', 'Rail', 'Sea']} for model_name in models_dict.keys()}

# Features (X) and target labels (y)
X = df.drop(columns=['Air', 'Road', 'Rail', 'Sea'])  # Define features
y_labels = ['Air', 'Road', 'Rail', 'Sea']  # Define target labels

# Create StratifiedKFold object for 5-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_models(models_dict, X, y_labels, skf, evaluation_reports, cv_f1_scores)

# Calculate mean F1 scores and output results
mean_f1_scores_df = calculate_mean_f1_scores(cv_f1_scores, y_labels)
print("\nMean F1 Scores Across Models and Labels:")
print(mean_f1_scores_df)

# Display classification reports as tables
generate_classification_report_tables(evaluation_reports)

# Example plot: Compare accuracy across models
plot_comparison('accuracy', evaluation_reports)

## Model Evaluation

In [ ]:
plot_comparison('f1-score', evaluation_reports) 

CatBoost consistently outperforms LightGBM and XGBoost across all labels, particularly for the minority classes (Class 1). This suggests that CatBoost might be better suited for handling class imbalance or more complex data distributions.

he Sea label poses the greatest challenge for all models, as indicated by the relatively lower F1 scores, especially for Class 1.
	•	Class 0 (likely the majority class) is consistently easier to classify across all labels and models, achieving high precision, recall, and F1 scores.


Accuracy:
- **Best Model**: CatBoost achieves the highest accuracy across the majority of labels, especially for “Air” (0.900), “Road” (0.905), and “Sea” (0.901).
- LightGBM and XGBoost perform similarly but slightly lag behind CatBoost in overall accuracy.
- CatBoost consistently outperforms LightGBM and XGBoost across all transport modes, especially for Sea transport, which appears to be the most challenging class.he differences between the models are small but meaningful, and CatBoost’s ability to handle categorical data and class imbalance without manual intervention gives it a slight advantage.

Cross-Validation:
- **Best Generalization**: CatBoost also demonstrates superior generalization with the highest cross-validation scores across all labels except “Sea.” This indicates that CatBoost is more consistent in making predictions across different data splits.
- XGBoost performs well for “Air” and “Rail,” while LightGBM is quite competitive but slightly less consistent for “Sea.”

CatBoost is the most reliable model for handling imbalanced datasets and can be favored for deployment.


## Model Optimization

In [ ]:
def plot_feature_importance(models_dict, feature_names):
    feature_importance_df = pd.DataFrame(index=feature_names)
    for model_name, model in models_dict.items():
        
        if hasattr(model, 'feature_importances_'):
            importance = model.feature_importances_
        else:
            importance = model.get_feature_importance()

        feature_importance_df[model_name] = importance
        indices = np.argsort(importance)[::-1]

        plt.figure(figsize=(10, 6))
        plt.title(f"{model_name} Feature Importance")
        plt.bar(range(len(feature_names)), importance[indices], align='center',color='blue', alpha=0.5)
        plt.xticks(range(len(feature_names)), np.array(feature_names)[indices], rotation=90)
        plt.tight_layout()
        plt.show()

    return feature_importance_df

feature_names = X.columns.tolist()
feature_importance_df = plot_feature_importance(models_dict, feature_names)
print("Feature Importance Values:")
print(feature_importance_df)

Insights:
	•	LightGBM: Feature importance in LightGBM is on a larger scale, which is normal for this model as the feature importance values are based on the number of times a feature is used in splits across all trees. For example:
	•	Net Weight: High importance with a score of 307.
	•	Width, Flammability Index, and Value are also quite important.
	•	XGBoost: The feature importance in XGBoost is expressed on a smaller scale (relative to LightGBM), as it reflects how useful a feature is in reducing the loss. Key features include:
	•	Value: The most important feature in XGBoost, with a score of 0.1136.
	•	Width and Packaging Cost also have significant importance.
	•	CatBoost: CatBoost calculates feature importance differently and focuses on how each feature impacts the model’s predictions. In CatBoost:
	•	Width: The most important feature, with a score of 13.28.
	•	Value, Packaging Cost, and Height are also crucial.
	Physical attributes (e.g., Net Weight, Length, Width, Height) and monetary value-related features (e.g., Value, Packaging Cost) are key drivers in predicting transport modes.
	•	CatBoost places higher importance on most features, particularly on dimensions and packaging costs, suggesting it captures more complex interactions between features.
	•	LightGBM has more variance in importance values, focusing strongly on a few key features like Width and Packaging Cost, while XGBoost distributes importance more evenly across features.

Conclusion:
	•	Common Important Features: Across all models, Value, Width, Packaging Cost, and Height consistently appear as important features, although their relative importance varies depending on the model.
	
The feature importance results suggest that most of the features are already contributing well to the model, especially in CatBoost. Removing lower-importance features like Storage or Size might marginally reduce model complexity but likely won’t bring significant performance improvements.

## Hyperparameter Tuning for CatBoost

In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

# Objective function for optimization
def objective(trial):
    # Define the hyperparameters for optimization
    param = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1, 10),
        'iterations': trial.suggest_int('iterations', 500, 1500),
        'border_count': trial.suggest_int('border_count', 32, 128),
        'random_strength': trial.suggest_loguniform('random_strength', 0.1, 10),
        'task_type': 'CPU',  # Or GPU if supported
        'verbose': 0
    }

    # Define the model
    catboost_model = CatBoostClassifier(**param)

    # Perform cross-validation and minimize negative F1 score
    f1 = cross_val_score(catboost_model, X, y, cv=3, scoring='f1_weighted').mean()
    return f1

# Define an Optuna study
study = optuna.create_study(direction='maximize')

# Start optimization
study.optimize(objective, n_trials=50)  # Run 50 trials

# Output the best parameters
print("Best parameters:", study.best_params)